In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

import numpy as np
import requests
import pandas as pd
from io import StringIO
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset

#load train data
import pandas as pd

import numpy as np
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import Dataset
import logging

from datasets import load_dataset
raw_datasets = load_dataset('json', data_files='https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/refs/heads/main/dataset/social_i_qa/train.json')

val_datasets = load_dataset('json', data_files='https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/refs/heads/main/dataset/social_i_qa/test.json')

/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
val_datasets

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'answer'],
        num_rows: 1954
    })
})

In [3]:
from sklearn.preprocessing import LabelEncoder



# Initialize and fit label encoder
label_encoder = LabelEncoder()
label_encoder.fit(raw_datasets['train']['answer'])
# Create the integer labels
train_labels = label_encoder.transform(raw_datasets['train']['answer'])

# Add a new 'labels' column
raw_datasets['train'] = raw_datasets['train'].add_column('labels', train_labels)

val_labels = label_encoder.transform(val_datasets['train']['answer'])

# Add a new 'labels' column
val_datasets['train'] = val_datasets['train'].add_column('labels', val_labels)


In [4]:
val_datasets['train']['answer']

['answer3',
 'answer1',
 'answer2',
 'answer1',
 'answer3',
 'answer1',
 'answer2',
 'answer2',
 'answer3',
 'answer2',
 'answer2',
 'answer1',
 'answer2',
 'answer1',
 'answer3',
 'answer2',
 'answer2',
 'answer2',
 'answer1',
 'answer1',
 'answer2',
 'answer2',
 'answer2',
 'answer2',
 'answer3',
 'answer1',
 'answer3',
 'answer2',
 'answer3',
 'answer1',
 'answer1',
 'answer2',
 'answer1',
 'answer2',
 'answer2',
 'answer3',
 'answer2',
 'answer2',
 'answer3',
 'answer3',
 'answer3',
 'answer2',
 'answer2',
 'answer2',
 'answer2',
 'answer1',
 'answer1',
 'answer3',
 'answer3',
 'answer3',
 'answer2',
 'answer1',
 'answer3',
 'answer1',
 'answer3',
 'answer1',
 'answer1',
 'answer3',
 'answer3',
 'answer1',
 'answer2',
 'answer3',
 'answer2',
 'answer3',
 'answer3',
 'answer1',
 'answer2',
 'answer2',
 'answer3',
 'answer1',
 'answer3',
 'answer1',
 'answer1',
 'answer1',
 'answer3',
 'answer1',
 'answer2',
 'answer1',
 'answer3',
 'answer3',
 'answer2',
 'answer1',
 'answer1',
 'an

In [5]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
#from roberta import RobertaForSequenceClassification


model_name = "microsoft/deberta-v3-base"

#config.num_labels=2
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'left'

/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:559: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [6]:
from datasets import DatasetDict

mask_token = tokenizer.mask_token

def generate_prompt(data_point):
    # sorry about the formatting disaster gotta move fast
    return f"""# input: {data_point["instruction"].split('format:')[0]}:{mask_token}"""
               


# Assuming `dataset` is your DatasetDict
def add_label_column(example):

    example['labels'] = example['labels']
  
    example['input'] = generate_prompt(example)

    
    return example

# Map the function over train and validation datasets

train_data = raw_datasets['train'].map(add_label_column)
val_data = val_datasets['train'].map(add_label_column)

# Remove unnecessary columns

# Inspect the updated datasets
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)

Train Dataset: Dataset({
    features: ['instruction', 'input', 'output', 'answer', 'labels'],
    num_rows: 33410
})
Validation Dataset: Dataset({
    features: ['instruction', 'input', 'output', 'answer', 'labels'],
    num_rows: 1954
})


In [7]:
train_data['input'][0]

'# input: Please choose the correct answer to the question: Cameron decided to have a barbecue and gathered her friends together. How would Others feel as a result?\n\nAnswer1: like attending Answer2: like staying home Answer3: a good friend to have\n\nAnswer :[MASK]'

In [8]:
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer.padding_side = 'left'


# col_to_delete = ['idx']
col_to_delete =  ['instruction', 'input', 'output', 'answer']

mask_token = tokenizer.mask_token
def preprocessing_function(examples):
   
    return tokenizer(examples['input'], truncation=True, max_length=512)

tokenized_train_data = train_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
tokenized_val_data = val_data.map(preprocessing_function, batched=True, remove_columns=col_to_delete)
# llama_tokenized_datasets = llama_tokenized_datasets.rename_column("target", "label")
tokenized_train_data.set_format("torch")
tokenized_val_data.set_format("torch")

# Data collator for padding a batch of examples to the maximum length seen in the batch
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [9]:
tokenizer.decode(tokenized_train_data['input_ids'][10])


'[CLS] # input: Please choose the correct answer to the question: Sydney was a school teacher and made sure their students learned well. How would you describe Sydney? Answer1: As someone that asked for a job Answer2: As someone that takes teaching seriously Answer3: Like a leader Answer :[MASK][SEP]'

In [10]:
val_data

Dataset({
    features: ['instruction', 'input', 'output', 'answer', 'labels'],
    num_rows: 1954
})

In [11]:
all_lengths = [len(ids) for ids in tokenized_train_data['input_ids']]
mx = max(all_lengths)
mx


176

In [12]:
count = sum(len(ids) > 512 for ids in tokenized_train_data['input_ids'])
print(count)


0


In [13]:
num_labels = len(label_encoder.classes_)
print(f"Total labels (classes): {num_labels}")


Total labels (classes): 3


In [14]:
import torch
import torch.nn as nn
from transformers import RobertaForSequenceClassification
from transformers.activations import ACT2FN
import random
from modeling import MLMSequenceClassification

config = AutoConfig.from_pretrained(model_name)

model = MLMSequenceClassification.from_pretrained(model_name, config=config, num_labels=num_labels, mask_token_id=tokenizer.mask_token_id)


Some weights of DebertaV2ForMaskedLM were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
import RoCoFT

RoCoFT.PEFT(model, method='column', rank=3) 
#targets=['key', 'value', 'dense', 'query'])

In [16]:

import evaluate
import numpy as np
from sklearn import metrics
import torch
import numpy as np

def compute_metrics(eval_pred):


    logits, labels = eval_pred # eval_pred is the tuple of predictions and labels returned by the model
    predictions = np.argmax(logits, axis=-1)
    
    precision = metrics.precision_score(labels, predictions, average="macro")
    recall = metrics.recall_score(labels, predictions, average="macro")
    f1 = metrics.f1_score(labels, predictions, average="macro")
    accuracy = metrics.accuracy_score(labels, predictions)
    
    return {"precision": precision, "recall": recall, "f1-score": f1, 'accuracy': accuracy}

In [17]:
from transformers import TrainingArguments, Trainer

import time
from transformers import Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir='dir',
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.0,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_total_limit=2,
    save_steps=10000000,
    logging_steps=100,
   
    load_best_model_at_end=True,
    lr_scheduler_type="cosine",  # You can choose from 'linear', 'cosine', 'cosine_with_restarts', 'polynomial', etc.
    warmup_steps=100,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_data,
    eval_dataset=tokenized_val_data,

    data_collator=data_collator,
    compute_metrics=compute_metrics
)

/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


[2025-04-28 11:40:43,746] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `__gxx_personality_v0@CXXABI_1.3'
/home/guangyu/anaconda3/envs/MD/compiler_compat/ld: /usr

In [18]:
trainer.train()

Got mask position:  tensor(-2, device='cuda:0')


Step,Training Loss,Validation Loss,Precision,Recall,F1-score,Accuracy
100,1.114700,1.107114,0.342712,0.333627,0.187013,0.329580
200,1.114100,1.105584,0.111566,0.333333,0.167178,0.334698
300,1.109800,1.109040,0.109690,0.333333,0.165062,0.329069
400,1.105700,1.099431,0.329972,0.335382,0.304918,0.334698
500,1.105100,1.100817,0.225850,0.330564,0.226273,0.332139
600,1.105200,1.099334,0.300828,0.324112,0.257522,0.325998
700,1.107300,1.101185,0.241783,0.340378,0.207851,0.341351
800,1.101900,1.100718,0.202071,0.331835,0.176444,0.334698
900,1.105400,1.098710,0.335203,0.327924,0.311808,0.329069
1000,1.100500,1.102134,0.213597,0.327011,0.249164,0.324463


/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/guangyu/anaconda3/envs/MD/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

TrainOutput(global_step=8356, training_loss=0.8914895836048117, metrics={'train_runtime': 1881.2191, 'train_samples_per_second': 71.039, 'train_steps_per_second': 4.442, 'total_flos': 62512865722368.0, 'train_loss': 0.8914895836048117, 'epoch': 4.0})

In [19]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import pickle
from transformers import AutoTokenizer, DataCollatorWithPadding


tokenizer.padding_side = 'left'

# Define preprocessing function with truncation
def preprocessing_function(examples):
    # Adjust 'text' to your input column name
    return tokenizer(
        examples['input'],
        padding=False,  # Padding handled by collator
        truncation=True,
        max_length=512  # Set to your model's max length
    )

# Apply preprocessing with truncation
col_to_delete=['instruction', 'input', 'output', 'answer']
tokenized_train_data1 = train_data.select(range(2000)).map(
    preprocessing_function,
    batched=True,
    remove_columns=col_to_delete
)

# Set data collator with explicit max_length
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="max_length",
    max_length=512  # Match tokenizer's max_length
)

# Create DataLoader
dataloader = DataLoader(
    tokenized_train_data1,
    batch_size=4,
    collate_fn=data_collator,
    shuffle=False
)

# Proceed with model evaluation and feature extraction
model.eval()
model.cuda()

X_list = []
Z_layer_outputs = [[] for _ in range(model.config.num_hidden_layers)]
Y_list = []

for batch in tqdm(dataloader):
    input_ids = batch['input_ids'].cuda()
    attention_mask = batch['attention_mask'].cuda()
    labels = batch['labels']
    
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, output_hidden_states=True)
        hidden_states = outputs.hidden_states
        
        # Store embeddings (mean pooled)
        X_list.append(hidden_states[0].mean(dim=1).cpu())
        
        # Store layer outputs (mean pooled)
        for i, layer_out in enumerate(hidden_states[1:]):
            Z_layer_outputs[i].append(layer_out.mean(dim=1).cpu())
        
        Y_list.append(labels)

# Stack all tensors
X_tensor = torch.cat(X_list, dim=0)
Z_tensors = [torch.cat(layer, dim=0) for layer in Z_layer_outputs]
Z_tensor = torch.stack(Z_tensors, dim=0)
Y_tensor = torch.cat(Y_list, dim=0)

# Save dataset
dataset_dict = {'X': X_tensor, 'Z': Z_tensor, 'Y': Y_tensor}
with open('full_dataset_hws_mlm.pkl', 'wb') as f:
    pickle.dump(dataset_dict, f)

100%|██████████| 500/500 [00:49<00:00, 10.14it/s]
